# CNN Visualization

Team:
    -Robbie Keehan
    -Max Goldstein
    -Spencer Chang

## Why not to use Deepdream 

When analyzing a CNN visually, there are a few techniques that give valuable insights into what the CNN has learnt. One of these is by starting with a picuture of random noise or a completely empty image. Then that image is fed through the already trained network and instead of updating the layers, the layer stays the same and the input image is iteratiely updated based off of gradient ascent. One appraoch to this is to see what the network expects a specific class to look like. This gives very valuable insights into what the network expects a class to look like, which can allow a human to decide whether or not the networks ideal version of a class is what it should be. Another approach is to do something similar, but see what the output is at every layer, which is called DeepDream. DeepDream offers artist value in creating weird morphed images, but it actually can provide helpful insights

In [138]:
from keras.applications.vgg16 import VGG16
from keras import backend as K

# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False)

In [160]:
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [192]:
def generate_pattern(layer_name, image, size=150):
    # Build a loss function that maximizes the activation
    # of the nth filter of the layer considered.
    
    layer_output = model.get_layer(layer_name).output
    loss = 0
    loss = (K.abs(layer_output))
    # Compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, model.input)[0]

    # Normalization trick: we normalize the gradient
    grads /= K.maximum(K.mean(K.abs(grads)),1e-7)

    # This function returns the loss and grads given the input picture
    iterate = K.function([model.input], [loss, grads])
    
    # We start from a random image selected
    input_img_data = image

    # Run gradient ascent for 40 steps
    step = .001
    for i in range(40):
        loss_value, grads_value = iterate([input_img_data])
#         print(loss_value)
        input_img_data += step * grads_value 
        
    img = input_img_data[0]
    return deprocess_image(img)

In [193]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [194]:
import scipy
def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

In [203]:
img_url = 'grass.jpeg'

# We preprocess the image into a 4D tensor
from keras.preprocessing import image
import numpy as np
import requests
from io import BytesIO
from PIL import Image

layer = 'block4_conv1'

def load_image_as_array(url):
#     response = requests.get(url)
    img = Image.open(img_url)
    img = np.array(img).astype(float)
    img = np.expand_dims(img, axis=0)
    img /= 255.
    return img

img = load_image_as_array(img_url)
shapes = img.shape[1:3]
new_shapes = [shapes]

for i in range (1,3):
    new_shape = tuple( [int(shapes[0]/(1.2*i)), int(shapes[1]/(1.2*i))])
    new_shapes.append(new_shape)
    
new_shapes = new_shapes[::-1]
print(new_shapes)
small_og_image = resize_img(img,new_shapes[0])
mega_ganglife_og = np.copy(img)
for idx,size in enumerate(new_shapes):
    # Save Last dreamed image
    if(idx+1 == len(new_shapes)):
        img = resize_img(img,size)
        img = generate_pattern(layer,img)
        img = img[np.newaxis,...].astype('float32')
        saved_img = Image.fromarray(img[0].astype('uint8'), 'RGB')
        print(saved_img)
        saved_img.save('dream{}.jpeg'.format(idx))
        continue
    # Resize Image Down
    img = resize_img(img,size)
    
    # Dream
    img = generate_pattern(layer,img)
    
    # add axis
    img = img[np.newaxis,...].astype('float32')
    
    # Save Image
    saved_img = Image.fromarray(img[0].astype('uint8'), 'RGB')
    saved_img.save('dream{}.jpeg'.format(idx))
    print(saved_img)
    
    # Upscale Dreamed Image
    img = resize_img(img,new_shapes[idx+1]) 
    
    # Upscale downscaled original image
    upsized_small_og = resize_img(small_og_image,new_shapes[idx+1]) # small image resized up
    
    # downscale original image
    downsize_og = resize_img(mega_ganglife_og,new_shapes[idx+1]) # original image resized down
    
    # find lost frequencies at given size
    lost_frequencies = downsize_og - upsized_small_og # lost detail
    # add back frequencies to dreamed image
    img += lost_frequencies 
    img /= 255.
   
        



import matplotlib.pyplot as plt



[(76, 114), (152, 229), (183, 275)]
<PIL.Image.Image image mode=RGB size=114x76 at 0x65690AA10>
<PIL.Image.Image image mode=RGB size=229x152 at 0x65B0822D0>
<PIL.Image.Image image mode=RGB size=275x183 at 0x657066710>


## Explaining model choice

## Implementing Deepdream

### Using L1 gradient normalization for gradient updates

### Adding random shifts/resizing in the process

## Testing and using deep dream process 

## Exceptional Credit: using another type of noise